#### Simple Gen AI APP Using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

/Users/yashsarjekar/Documents/AI_WORK/Langchain/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader=WebBaseLoader("https://github.com/yashsarjekar")
loader

In [4]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://github.com/yashsarjekar', 'title': 'yashsarjekar (Yash Sarjekar) · GitHub', 'description': 'yashsarjekar has 35 repositories available. Follow their code on GitHub.', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nyashsarjekar (Yash Sarjekar) · GitHub\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu\n\nToggle navigation\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n            Sign in\n          \n\n\n \n\n\nAppearance settings\n\n\n\n\n\n\n\n\n\n\n\nPlatformAI CODE CREATIONGitHub CopilotWrite better code with AIGitHub SparkBuild and deploy intelligent appsGitHub ModelsManage and compare promptsMCP RegistryNewIntegrate external toolsDEVELOPER WORKFLOWSActionsAutomate any workflowCodespacesInstant dev environmentsIssuesPlan and track workCode ReviewManage code 

In [5]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [6]:
documents

[Document(metadata={'source': 'https://github.com/yashsarjekar', 'title': 'yashsarjekar (Yash Sarjekar) · GitHub', 'description': 'yashsarjekar has 35 repositories available. Follow their code on GitHub.', 'language': 'en'}, page_content='yashsarjekar (Yash Sarjekar) · GitHub\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu\n\nToggle navigation\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n            Sign in\n          \n\n\n \n\n\nAppearance settings'),
 Document(metadata={'source': 'https://github.com/yashsarjekar', 'title': 'yashsarjekar (Yash Sarjekar) · GitHub', 'description': 'yashsarjekar has 35 repositories available. Follow their code on GitHub.', 'language': 'en'}, page_content='PlatformAI CODE CREATIONGitHub CopilotWrite better code with AIGitHub SparkBuild and deploy intelligent appsGitHub ModelsManage and compare promptsMCP RegistryNewIntegrate external toolsDEVELOPER WORKFL

In [7]:
len(documents)

11

In [8]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [10]:
from langchain_chroma import Chroma
vectordb=Chroma.from_documents(documents=documents,embedding=embeddings)
vectordb

In [11]:
vectordb

In [13]:
## Query From a vector db
query="Full Stack Engineer with 5+ years of experience building"
result=vectordb.similarity_search(query)
result[0].page_content

'🛠️ Tech Stack\n\nLanguages: Python, JavaScript, TypeScript, SQL, HTML5, CSS3\nBackend: Django, Flask, FastAPI, Node.js\nFrontend: Tailwind, HTMX, React, Bootstrap\nDatabases: PostgreSQL, Redis, MySQL, MongoDB\nCloud: AWS (EC2, S3, RDS, Lambda), Azure\nDevOps: Docker, Jenkins, CI/CD, Git, Linux\nTesting: Pytest, Unit Testing, Code Reviews\nAPIs: REST API, GraphQL, Django REST Framework\n\n\n💼 Experience\nSenior Software Engineer\nTwisted Mountain Animation | Jan 2025 – Present\n\nDesigned and deployed features using Django + PostgreSQL.\nOptimized SQL queries → 40% faster response times.\nImplemented Redis caching for high-traffic apps.\nCollaborated with international product & design teams.\n\nSoftware Development Engineer\nClickpost | Apr 2023 – Jan 2025\n\nBuilt scalable e-commerce solutions handling 30K+ daily transactions.\nImproved system efficiency by 40% via SQL + Redis optimizations.\nDeveloped responsive UIs using Tailwind + JS frameworks.\nManaged Dockerized deployments on 

In [14]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [20]:
## Retrieval Chain, Document chain

from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.co

In [17]:
pip show langchain


Name: langchain
Version: 1.2.0
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /Users/yashsarjekar/Documents/AI_WORK/Langchain/venv/lib/python3.13/site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [21]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

'LangSmith has two usage limits: total traces and extended traces. These are the two metrics tracked on their usage graph.'

In [24]:
### Input--->Retriever--->vectorstoredb

vectordb

In [25]:
retriever=vectordb.as_retriever()
from langchain_classic.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [26]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x29ec70050>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | ChatOpenAI(pro

In [27]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"Full Stack Engineer with 5+ years of experience "})
response['answer']

'Based on the provided context, here is a summarized answer:\n\nYash Sarjekar is a Full Stack Engineer with over 5 years of experience, specializing in building scalable web applications using Django and PostgreSQL, alongside modern frontend frameworks. He has worked as a Senior Software Engineer at Twisted Mountain Animation since January 2025, focusing on designing and deploying features, optimizing SQL queries, and implementing Redis caching. Previously, he was a Software Development Engineer at Clickpost, where he built scalable e-commerce solutions and managed Dockerized deployments on AWS. His education includes a Bachelor of Engineering in Information Technology from the University of Mumbai.\n\nKey strengths include Django and PostgreSQL expertise, SQL query optimization, Redis performance tuning, and full-stack development using Tailwind, HTMX, and JavaScript. He holds certifications in AWS Solution Architecture and IBM Data Science, and has a strong skill set in DevOps and cl

In [28]:
response

{'input': 'Full Stack Engineer with 5+ years of experience ',
 'context': [Document(id='b8589d81-c7a8-436e-813f-8e67ea531e68', metadata={'description': 'yashsarjekar has 35 repositories available. Follow their code on GitHub.', 'title': 'yashsarjekar (Yash Sarjekar) · GitHub', 'language': 'en', 'source': 'https://github.com/yashsarjekar'}, page_content='🛠️ Tech Stack\n\nLanguages: Python, JavaScript, TypeScript, SQL, HTML5, CSS3\nBackend: Django, Flask, FastAPI, Node.js\nFrontend: Tailwind, HTMX, React, Bootstrap\nDatabases: PostgreSQL, Redis, MySQL, MongoDB\nCloud: AWS (EC2, S3, RDS, Lambda), Azure\nDevOps: Docker, Jenkins, CI/CD, Git, Linux\nTesting: Pytest, Unit Testing, Code Reviews\nAPIs: REST API, GraphQL, Django REST Framework\n\n\n💼 Experience\nSenior Software Engineer\nTwisted Mountain Animation | Jan 2025 – Present\n\nDesigned and deployed features using Django + PostgreSQL.\nOptimized SQL queries → 40% faster response times.\nImplemented Redis caching for high-traffic apps.\

In [29]:
response.keys()

dict_keys(['input', 'context', 'answer'])

In [30]:
response['context']

[Document(id='b8589d81-c7a8-436e-813f-8e67ea531e68', metadata={'description': 'yashsarjekar has 35 repositories available. Follow their code on GitHub.', 'title': 'yashsarjekar (Yash Sarjekar) · GitHub', 'language': 'en', 'source': 'https://github.com/yashsarjekar'}, page_content='🛠️ Tech Stack\n\nLanguages: Python, JavaScript, TypeScript, SQL, HTML5, CSS3\nBackend: Django, Flask, FastAPI, Node.js\nFrontend: Tailwind, HTMX, React, Bootstrap\nDatabases: PostgreSQL, Redis, MySQL, MongoDB\nCloud: AWS (EC2, S3, RDS, Lambda), Azure\nDevOps: Docker, Jenkins, CI/CD, Git, Linux\nTesting: Pytest, Unit Testing, Code Reviews\nAPIs: REST API, GraphQL, Django REST Framework\n\n\n💼 Experience\nSenior Software Engineer\nTwisted Mountain Animation | Jan 2025 – Present\n\nDesigned and deployed features using Django + PostgreSQL.\nOptimized SQL queries → 40% faster response times.\nImplemented Redis caching for high-traffic apps.\nCollaborated with international product & design teams.\n\nSoftware Devel